In [1]:
#@title Install required packages
!uv pip uninstall "." # proxy structuring engine
!uv pip install -U ".[dev]" # proxy structuring engine
!uv pip install sentencepiece
!uv pip install accelerate
!uv pip install transformers
!uv pip install bitsandbytes
!uv pip install sentencepiece
!uv pip install protobuf
!uv pip install -U tqdm
!uv pip install ipywidgets

Uninstalled 1 package in 2ms
 - pse==1.5.9 (from file:///Users/jckwind/Documents/proxy-structuring-engine)
Resolved 73 packages in 529ms                                        
Building pse @ file:///Users/jckwind/Documents/proxy-structuring-engine
Building pse @ file:///Users/jckwind/Documents/proxy-structuring-engine
   Built pse @ file:///Users/jckwind/Documents/proxy-structuring-engine
Prepared 1 package in 240ms                                              
Installed 1 package in 1msle:///Users/jckwind/Documents/prox
 + pse==1.5.9 (from file:///Users/jckwind/Documents/proxy-structuring-engine)
Audited 1 package in 1ms
Audited 1 package in 2ms
Audited 1 package in 2ms
Audited 1 package in 1ms
Audited 1 package in 1ms
Audited 1 package in 1ms
Resolved 1 package in 56ms                                           
Audited 1 package in 0.09ms
Audited 1 package in 1ms


In [2]:
# @title Setup Llama 3.2 1B and create the structuring engine
import torch
from transformers import (
    AutoTokenizer,
    LlamaForCausalLM,
    LogitsProcessorList,
)

from pse.engine.structuring_engine import StructuringEngine
from pse.util.torch_mixin import PSE_TorchMixin


class PSE_Torch(PSE_TorchMixin, LlamaForCausalLM):
    pass


model_path = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = PSE_Torch.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

model.engine = StructuringEngine(tokenizer)
model.config.pad_token_id = model.config.eos_token_id[0]
if model.generation_config:
    model.generation_config.pad_token_id = model.config.eos_token_id[0]

In [3]:

#@title Test json generation
import json

SIMPLE_JSON_SCHEMA = {
    "type": "object",
    "properties": {"value": {"type": "number"}},
    "required": ["value"],
}
model.engine.configure(SIMPLE_JSON_SCHEMA)
prompt = "Please generate a json object with the value 9.11, with the following schema:\n"
prompt += json.dumps(SIMPLE_JSON_SCHEMA, indent=2)
messages = [{"role": "user", "content": prompt}]
input_ids = tokenizer.apply_chat_template(
    messages, return_tensors="pt", add_generation_prompt=True
)
assert isinstance(input_ids, torch.Tensor)
input_ids = input_ids.to(model.device)
assert isinstance(input_ids, torch.Tensor)
output = model.generate(
    input_ids,
    do_sample=True,
    temperature=1.0,
)
print("Output:\n" + 100 * "-")
print(tokenizer.decode(output[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Output:
----------------------------------------------------------------------------------------------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 12 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Please generate the number 9.11 in a json object with the following schema:
{'type': 'object', 'properties': {'value': {'type': 'number'}},'required': ['value']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{
  "value": 9.11
}
